# Clustering: images

## Contents

- [1. Preliminary](#1-preliminary)
    - [1.1 Imports](#1-1-import)
    - [1.2 Initial data preparations](#1-2-init)
- [2. Original data clustering](#2-orig)
    - [2.1 Initialization](#2-1-init)
    - [2.2 Clustering](#2-2-clust)
    - [2.3 Results](#2-3-results)
- [3. Common orthogonal basis extraction (COBE)](#3-cobe)
    - [3.1 Initialization](#3-1-init)
    - [3.2 Computations](#3-2-comp)
    - [3.3 Clustering](#3-3-clust)
    - [3.4 Results](#3-4-results)
- [4. Group independent component analysis (GICA)](#4-gica)
    - [4.1 Initialization](#4-1-init)
    - [4.2 Computations](#4-2-comp)
    - [4.3 Clustering](#4-3-clust)
    - [4.4 Results](#4-4-results)
- [5. Group (Lr, 1) decomposition (GLRO)](#5-glro)
    - [5.1 Initialization](#5-1-init)
    - [5.2 Computations](#5-2-comp)
    - [5.3 Clustering](#5-3-clust)
    - [5.4 Results](#5-4-results)
- [6. Group Tucker-(Lr, 1) decomposition (GTLD)](#6-gtld)
    - [6.1 Initialization](#6-1-init)
    - [6.2 Computations](#6-2-comp)
    - [6.3 Clustering](#6-3-clust)
    - [6.4 Results](#6-4-results)


## 1. Preliminary <a class="anchor" id="1-preliminary"></a>
    
- [1.1 Imports](#1-1-import)
- [1.2 Initial data preparations](#1-2-init)


[Back to contents](#Contents)

### 1.1 Imports <a class="anchor" id="1-1-import"></a>

[Up to section](#1-preliminary) $\quad$
[Back to contents](#Contents)

In [ ]:
_NUM_THREADS = 2
%env OMP_NUM_THREADS=_NUM_THREADS

import numpy as np
# import ctypes
# mkl_rt = ctypes.CDLL('libmkl_rt.so')
# mkl_get_max_threads = mkl_rt.mkl_get_max_threads
# def mkl_set_num_threads(cores):
#     mkl_rt.mkl_set_num_threads(ctypes.byref(ctypes.c_int(cores)))
# mkl_set_num_threads(_NUM_THREADS)
# print mkl_get_max_threads() # says 4

import sys
sys.path.append('..')
import os
import copy
import time

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score

from gtcd import reshape
#from computational_utilities import reshape

from cluster.cobe_contrasting import COBEContrast
from cluster.gica_contrasting import GICAContrast
from cluster.gtcd_contrasting import GLROContrast
from cluster.gtcd_contrasting import GTLDContrast

from sklearn.cluster import AgglomerativeClustering

# clustering performance measures
from sklearn.metrics import fowlkes_mallows_score
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import normalized_mutual_info_score
from sklearn.metrics import adjusted_mutual_info_score

from sklearn.metrics import pairwise_distances

### 1.2 Initial data preparations <a class="anchor" id="1-2-init"></a>

[Up to section](#1-preliminary) $\quad$
[Back to contents](#Contents)

In [2]:
data_dirname = '../data/'
data_filename = '../data/eth80_128.npz'

df = np.load(data_dirname+data_filename)

data, labels = df['data'], df['classes']
labels = labels.tolist()
Nclasses, Nobjects, Nframes, Npix1, Npix2, Ncolors = data.shape
Npix = Npix1*Npix2
Nsamples = Nclasses*Nobjects
labels = labels*Nobjects

new_shape = [-1, Nframes, Npix, Ncolors]
data = reshape(data, new_shape)

classEncoder = LabelEncoder()
classEncoder.fit(labels)
y = classEncoder.transform(labels)
#classEncoder.inverse_transform(y)

random_state = 235
n_splits = 4
n_repeats = 1
separator = RepeatedStratifiedKFold(
    n_splits=n_splits, n_repeats=n_repeats, random_state=random_state
)

dummyX = np.empty(Nsamples)
train_indices, test_indices = [], []
for train_index, test_index in separator.split(dummyX, y):
    train_indices.append(train_index)
    test_indices.append(test_index)

n_clusters = len(np.unique(labels))

## 2. Original data clustering <a class="anchor" id="2-orig"></a>

- [2.1 Initialization](#2-1-init)
- [2.2 Clustering](#2-2-clust)
- [2.3 Results](#2-3-results)

[Back to contents](#Contents)

### 2.1 Initialization <a class="anchor" id="2-1-init"></a>

[Up to section](#2-orig) $\quad$
[Back to contents](#Contents)

In [ ]:
transposition = [0, 2, 1, 3]
X = np.transpose(data, transposition)
shapePlain = [Nsamples, Npix*Nframes*Ncolors]
X = reshape(X, shapePlain)
indices = copy.deepcopy(train_indices)

affinity_names = [
    'l1', 'l2', 'cosine', 'canberra', 'correlation', 'rbf'
]
linkage = [
    'complete', 'average'
]


### 2.2 Clustering <a class="anchor" id="2-2-clust"></a>

[Up to section](#2-orig) $\quad$
[Back to contents](#Contents)

In [ ]:
save_results_dirname = '../results/clustering/'
save_results_filename = 'plain_ETH80_ac'

try:
    os.makedirs(save_results_dirname)
except:
    print 'Already exist: %s' % (save_results_dirname)

result = np.zeros([n_splits, len(affinity_names), len(linkage), 3])

clustAlg = AgglomerativeClustering(n_clusters=n_clusters)
for k_ind in xrange(n_splits):
    print 'Index %d / %d' % (k_ind+1, len(indices))
    current_index = indices[k_ind]
    T = X[current_index].copy()
    for k_affinity in xrange(len(affinity_names)):
        current_affinity = affinity_names[k_affinity]
        for k_linkage in xrange(len(linkage)):
            current_linkage = linkage[k_linkage]
            clustAlg.linkage = current_linkage
            if ((current_affinity == 'canberra') or 
                (current_affinity == 'correlation') or
                (current_affinity == 'rbf')):
                clustAlg.affinity = 'precomputed'
                if current_affinity == 'rbf':
                    D = pairwise_distances(T, metric='euclidean')
                    D = - np.exp(-D)
                else:
                    D = pairwise_distances(T, metric=current_affinity)
                pred = clustAlg.fit_predict(D)
            else:
                clustAlg.affinity = current_affinity
                pred = clustAlg.fit_predict(T)
            result[k_ind, k_affinity, k_linkage, 0] = adjusted_rand_score(
                y[indices[k_ind]],
                pred
            )
            result[k_ind, k_affinity, k_linkage, 1] = adjusted_mutual_info_score(
                y[indices[k_ind]],
                pred
            )
            result[k_ind, k_affinity, k_linkage, 2] = fowlkes_mallows_score(
                y[indices[k_ind]],
                pred
            )
            np.savez_compressed(
                save_results_dirname+save_results_filename, result=result, affinity_names=affinity_names,
                indices=indices, linkage=linkage
            )

### 2.3 Results <a class="anchor" id="2-3-results"></a>

[Up to section](#2-orig) $\quad$
[Back to contents](#Contents)

In [4]:
results_dirname = '../results/clustering/'
results_filename = 'plain_ETH80_ac.npz'

df = np.load(results_dirname + results_filename)

linkageList = df['linkage']
affinityList = df['affinity_names']

a = np.mean(df['result'], axis=0)[:, :]
maxval = a[:, :, 0].max()
maxind = np.where(a[:, :, 0] == maxval)
print maxval, maxind
print linkageList[maxind[1][0]], affinityList[maxind[0][0]]
v1, v2, v3 = a[maxind[0][0], maxind[1][0]]
print " ARI  AMI  FMS"
print "%.3f %.3f %.3f" % (v1, v2, v3)
print linkageList[0], affinityList[3]
v1, v2, v3 = a[3, 0]
print "%.3f %.3f %.3f" % (v1, v2, v3)

0.544705000668 (array([4]), array([1]))
average correlation
 ARI  AMI  FMS
0.545 0.669 0.615
complete canberra
0.423 0.556 0.509


## 3. Common orthogonal basis extraction (COBE) <a class="anchor" id="3-cobe"></a>

- [3.1 Initialization](#3-1-init)
- [3.2 Computations](#3-2-comp)
- [3.3 Clustering](#3-3-clust)
- [3.4 Results](#3-4-results)

[Back to contents](#Contents)

### 3.1 Initialization <a class="anchor" id="3-1-init"></a>

[Up to section](#3-cobe) $\quad$
[Back to contents](#Contents)

In [5]:
transposition = [0, 2, 1, 3]
X = np.transpose(data, transposition)
shapeCOBE = [Nsamples, Npix, Nframes*Ncolors]
X = reshape(X, shapeCOBE)
indices = copy.deepcopy(train_indices)
commonRanks = 1+np.arange(10)   

affinity_names = [
    'l1', 'l2', 'cosine', 'canberra', 'correlation', 'rbf'
]
linkage = [
    'complete', 'average'
]


### 3.2 Computations <a class="anchor" id="3-2-comp"></a>

[Up to section](#3-cobe) $\quad$
[Back to contents](#Contents)

In [ ]:
save_model_dirname = '../models/clustering/cobe/'
save_filename_base = 'COBE_contrasted_ETH80'

try:
    os.makedirs(save_model_dirname)
except:
    print 'Already exist: %s' % (save_model_dirname)

times = np.zeros([len(commonRanks), len(indices)])

for k_comr in xrange(len(commonRanks)):
    commonRank = commonRanks[k_comr]
    print '\t %d rank' % (commonRank)
    for k_ind in xrange(n_splits):
        print 'Index %d / %d' % (k_ind+1, len(indices))
        current_index = indices[k_ind]
        t1 = time.clock()
        contrasting = COBEContrast(
            n_clusters, commonRank, shapeObject=shapeCOBE[1:], maxitnum=100, epsilon=1e-5
        )
        t2 = time.clock()
        times[k_comr, k_ind] = t2-t1
        T = contrasting.fit_transform(X[current_index], commonRank)
        save_filename = str(k_ind) + '_' + save_filename_base + '_crank='+str(commonRank)
        np.savez_compressed(save_model_dirname+save_filename, T=T, times=times)

### 3.3 Clustering <a class="anchor" id="3-3-clust"></a>

[Up to section](#3-cobe) $\quad$
[Back to contents](#Contents)

In [ ]:
model_dirname = '../models/clustering/cobe/'
model_filename_base = 'COBE_contrasted_ETH80'

save_results_dirname = '../results/clustering/'
save_results_filename = 'cobe_ETH80_ac'

try:
    os.makedirs(save_results_dirname)
except:
    print 'Already exist: %s' % (save_results_dirname)


result = np.zeros([len(commonRanks), n_splits, len(affinity_names), len(linkage), 3])
clustAlg = AgglomerativeClustering(n_clusters=n_clusters)

for k_comr in xrange(len(commonRanks)):
    commonRank = commonRanks[k_comr]
    print '\t %d rank' % (commonRank)
    for k_ind in xrange(n_splits):#len(indices)):
        print 'Index %d / %d' % (k_ind+1, len(indices))
        current_index = indices[k_ind]
        current_df = np.load(
            model_dirname+str(k_ind)+'_'+model_filename_base+'_crank='+str(commonRank)+\
            '.npz'
        )
        T = current_df['T']
        T = reshape(T, [T.shape[0], -1])
        for k_affinity in xrange(len(affinity_names)):
            current_affinity = affinity_names[k_affinity]
            for k_linkage in xrange(len(linkage)):
                current_linkage = linkage[k_linkage]
                clustAlg.linkage = current_linkage
                if ((current_affinity == 'canberra') or 
                    (current_affinity == 'correlation') or
                    (current_affinity == 'rbf')):
                    clustAlg.affinity = 'precomputed'
                    if current_affinity == 'rbf':
                        D = pairwise_distances(T, metric='euclidean')
                        D = - np.exp(-D)
                    else:
                        D = pairwise_distances(T, metric=current_affinity)
                    pred = clustAlg.fit_predict(D)
                else:
                    clustAlg.affinity = current_affinity
                    pred = clustAlg.fit_predict(T)
                result[k_comr, k_ind, k_affinity, k_linkage, 0] = adjusted_rand_score(
                    y[indices[k_ind]],
                    pred
                )
                result[k_comr, k_ind, k_affinity, k_linkage, 1] = adjusted_mutual_info_score(
                    y[indices[k_ind]],
                    pred
                )
                result[k_comr, k_ind, k_affinity, k_linkage, 2] = fowlkes_mallows_score(
                    y[indices[k_ind]],
                    pred
                )
                np.savez_compressed(
                    save_results_dirname+save_results_filename, result=result,
                    affinity_names=affinity_names, indices=indices,
                    linkage=linkage
                )


### 3.4 Results <a class="anchor" id="3-4-results"></a>

[Up to section](#3-cobe) $\quad$
[Back to contents](#Contents)

In [7]:
results_dirname = '../results/clustering/'
results_filename = 'cobe_ETH80_ac.npz'

df = np.load(results_dirname+results_filename)

linkageList = df['linkage']
affinityList = df['affinity_names']
commonRanks = 1+np.arange(a.shape[0])

a = np.mean(df['result'], axis=1)
maxval = a[:, :, :, 0].max()
maxind = np.where(a[:, :, :, 0] == maxval)
print maxval, maxind
print linkageList[maxind[2][0]], affinityList[maxind[1][0]], commonRanks[maxind[0][0]]
v1, v2, v3 = a[maxind[0][0], maxind[1][0], maxind[2][0]]
print " ARI  AMI  FMS"
print "%.3f %.3f %.3f" % (v1, v2, v3)

 0.692952179571 (array([8]), array([3]), array([0]))
complete canberra 9
 ARI  AMI  FMS
0.693 0.775 0.732


## 4. Group independent component analysis (GICA) <a class="anchor" id="4-gica"></a>

- [4.1 Initialization](#4-1-init)
- [4.2 Computations](#4-2-comp)
- [4.3 Clustering](#4-3-clust)
- [4.4 Results](#4-4-results)

[Back to contents](#Contents)

### 4.1 Initialization <a class="anchor" id="4-1-init"></a>

[Up to section](#4-gica) $\quad$
[Back to contents](#Contents)

In [8]:
transposition = [0, 2, 1, 3]
X = np.transpose(data, transposition)
shapeGICA = [Nsamples, Npix, Nframes*Ncolors]
X = reshape(X, shapeGICA)
indices = copy.deepcopy(train_indices)
commonRanks = 1+np.arange(10)
individualRanks = 1+np.arange(10)

affinity_names = [
    'l1', 'l2', 'cosine', 'canberra', 'correlation', 'rbf'
]
linkage = [
    'complete', 'average'
]


### 4.2 Computations <a class="anchor" id="4-2-comp"></a>

[Up to section](#4-gica) $\quad$
[Back to contents](#Contents)

In [ ]:
save_model_dirname = '../models/clustering/gica/'
save_filename_base = 'GICA_contrasted_ETH80'

try:
    os.makedirs(save_model_dirname)
except:
    print 'Already exist: %s' % (save_model_dirname)

times = np.zeros([len(commonRanks), len(individualRanks), len(indices)])

for k_comr in xrange(len(commonRanks)):
    commonRank = commonRanks[k_comr]
    print '\t %d rank' % (commonRank)
    for k_indr in xrange(len(individualRanks)):
        individualRank = individualRanks[k_indr]
        print '\t\t %d rank' % (individualRank)
        for k_ind in xrange(n_splits):
            print 'Index %d / %d' % (k_ind+1, len(indices))
            current_index = indices[k_ind]
            t1 = time.clock()
            contrasting = GICAContrast(
                n_clusters, individualRank, commonRank, shapeObject=shapeGICA[1:],
                maxitnum=100, epsilon=1e-5
            )
            t2 = time.clock()
            times[k_comr, k_ind] = t2-t1
            T = contrasting.fit_transform(X[current_index], individualRank, commonRank)
            save_filename = str(k_ind) + '_' + save_filename_base + '_crank='+str(commonRank)
            save_filename += '_irank='+str(individualRank)
            np.savez_compressed(save_model_dirname+save_filename, T=T, times=times)

### 4.3 Clustering <a class="anchor" id="4-3-clust"></a>

[Up to section](#4-gica) $\quad$
[Back to contents](#Contents)

In [ ]:
model_dirname = '../models/clustering/gica/'
model_filename_base = 'GICA_contrasted_ETH80'

save_results_dirname = '../results/clustering/'
save_results_filename = 'gica_ETH80_ac'

try:
    os.makedirs(save_results_dirname)
except:
    print 'Already exist: %s' % (save_results_dirname)


result = np.zeros([len(commonRanks), len(individualRanks), n_splits, len(affinity_names), len(linkage), 3])
clustAlg = AgglomerativeClustering(n_clusters=n_clusters)

for k_comr in xrange(len(commonRanks)):
    commonRank = commonRanks[k_comr]
    print '\t %d rank' % (commonRank)
    for k_indr in xrange(len(individualRanks)):
        individualRank = individualRanks[k_indr]
        print '\t\t %d rank' % (individualRank)
        for k_ind in xrange(n_splits):#len(indices)):
            print 'Index %d / %d' % (k_ind+1, len(indices))
            current_index = indices[k_ind]
            current_df = np.load(
                model_dirname+str(k_ind)+'_'+model_filename_base+'_crank='+str(commonRank)+\
                '_irank='+str(individualRank)+'.npz'
            )
            T = current_df['T']
            T = reshape(T, [T.shape[0], -1])
            for k_affinity in xrange(len(affinity_names)):
                current_affinity = affinity_names[k_affinity]
                for k_linkage in xrange(len(linkage)):
                    current_linkage = linkage[k_linkage]
                    clustAlg.linkage = current_linkage
                    if ((current_affinity == 'canberra') or 
                        (current_affinity == 'correlation') or
                        (current_affinity == 'rbf')):
                        clustAlg.affinity = 'precomputed'
                        if current_affinity == 'rbf':
                            D = pairwise_distances(T, metric='euclidean')
                            D = - np.exp(-D)
                        else:
                            D = pairwise_distances(T, metric=current_affinity)
                        pred = clustAlg.fit_predict(D)
                    else:
                        clustAlg.affinity = current_affinity
                        pred = clustAlg.fit_predict(T)
                    result[k_comr, k_indr, k_ind, k_affinity, k_linkage, 0] = adjusted_rand_score(
                        y[indices[k_ind]],
                        pred
                    )
                    result[k_comr, k_indr, k_ind, k_affinity, k_linkage, 1] = adjusted_mutual_info_score(
                        y[indices[k_ind]],
                        pred
                    )
                    result[k_comr, k_indr, k_ind, k_affinity, k_linkage, 2] = fowlkes_mallows_score(
                        y[indices[k_ind]],
                        pred
                    )
                    np.savez_compressed(
                        save_results_dirname+save_results_filename, result=result,
                        affinity_names=affinity_names, indices=indices, linkage=linkage
                    )


### 4.4 Results <a class="anchor" id="4-4-results"></a>

[Up to section](#4-gica) $\quad$
[Back to contents](#Contents)

In [19]:
results_dirname = '../results/clustering/'
results_filename = 'gica_ETH80_ac.npz'

df = np.load(results_dirname+results_filename)
linkageList = df['linkage']
affinityList = df['affinity_names']
commonRanks = 1+np.arange(a.shape[0])
individualRanks = 1+np.arange(a.shape[1])

a = np.mean(df['result'], axis=2)
maxval = a[:, :, :, :, 0].max()
maxind = np.where(a[:, :, :, :, 0] == maxval)
print maxval, maxind

print (
    linkageList[maxind[3][0]], affinityList[maxind[2][0]],
    commonRanks[maxind[0][0]], individualRanks[maxind[1][0]]
)


v1, v2, v3 = a[maxind[0][0], maxind[1][0], maxind[2][0], maxind[3][0]]
print " ARI  AMI  FMS"
print "%.3f %.3f %.3f" % (v1, v2, v3)

0.614945400081 (array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9]), array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
('complete', 'canberra', 10, 1)
 ARI  AMI  FMS
0.615 0.728 0.669


## 5. Group (Lr, 1) decomposition (GLRO) <a class="anchor" id="5-glro"></a>

- [5.1 Initialization](#5-1-init)
- [5.2 Computations](#5-2-comp)
- [5.3 Clustering](#5-3-clust)
- [5.4 Results](#5-4-results)

[Back to contents](#Contents)

### 5.1 Initialization <a class="anchor" id="5-1-init"></a>

[Up to section](#5-glro) $\quad$
[Back to contents](#Contents)

In [14]:
transposition = [0, 2, 1, 3]
X = np.transpose(data, transposition)
shapeGLRO = [Nsamples, Npix, Nframes, Ncolors]
X = reshape(X, shapeGLRO)
indices = copy.deepcopy(train_indices)
commonRanks = 1+np.arange(10)
individualRanks = 1+np.arange(10)
sourceModes = [0]

affinity_names = [
    'l1', 'l2', 'cosine', 'canberra', 'correlation', 'rbf'
]
linkage = [
    'complete', 'average'
]

method_name = 'als'
constraint_method = 'projected'
maxitnum = 10

### 5.2 Computations <a class="anchor" id="5-2-comp"></a>

[Up to section](#5-glro) $\quad$
[Back to contents](#Contents)

In [ ]:
save_model_dirname = '../models/clustering/glro/'
save_filename_base = 'GLRO_contrasted_ETH80'

try:
    os.makedirs(save_model_dirname)
except:
    print 'Already exist: %s' % (save_model_dirname)

times = np.zeros([len(commonRanks), len(individualRanks), len(indices)])
for k_comr in xrange(len(commonRanks)):
    commonRank = commonRanks[k_comr]
    print '\t %d rank' % (commonRank)
    for k_indr in xrange(len(individualRanks)):
        individualRank = individualRanks[k_indr]
        print '\t\t %d rank' % (individualRank)
        for k_ind in xrange(n_splits):
            print 'Index %d / %d' % (k_ind+1, len(indices))
            current_index = indices[k_ind]
            
            contrasting = GLROContrast(
                n_clusters, individualRank, commonRank, shapeObject=shapeGLRO[1:],
                maxitnum=maxitnum, epsilon=1e-5, sourceModes=sourceModes,
                method=method_name, nShortModes=None, constraintMethod=constraint_method,
                fullModesConstraint=None
            )
            
            t1 = time.clock()
            T = contrasting.fit_transform(
                X[current_index], individualRank, commonRank, 
                 verbose=True, recover=True
            )
            t2 = time.clock()
            times[k_comr, k_ind] = t2-t1
            save_filename = str(k_ind) + '_' + save_filename_base + '_crank='+str(commonRank)
            save_filename += '_irank='+str(individualRank)
            np.savez_compressed(save_model_dirname+save_filename, T=T, times=times)

### 5.3 Clustering <a class="anchor" id="5-3-clust"></a>

[Up to section](#5-glro) $\quad$
[Back to contents](#Contents)

In [ ]:
model_dirname = '../models/clustering/glro/'
model_filename_base = 'GLRO_contrasted_ETH80'

save_results_dirname = '../results/clustering/'
save_results_filename = 'glro_ETH80_ac'

try:
    os.makedirs(save_results_dirname)
except:
    print 'Already exist: %s' % (save_results_dirname)

result = np.zeros([len(commonRanks), len(individualRanks), n_splits, len(affinity_names), len(linkage), 3])
clustAlg = AgglomerativeClustering(n_clusters=n_clusters)

for k_comr in xrange(len(commonRanks)):
    commonRank = commonRanks[k_comr]
    print '\t %d rank' % (commonRank)
    for k_indr in xrange(len(individualRanks)):
        individualRank = individualRanks[k_indr]
        print '\t\t %d rank' % (individualRank)
        for k_ind in xrange(n_splits):#len(indices)):
            print 'Index %d / %d' % (k_ind+1, len(indices))
            current_index = indices[k_ind]
            current_df = np.load(
                model_dirname+str(k_ind)+'_'+model_filename_base+'_crank='+str(commonRank)+\
                '_irank='+str(individualRank)+'.npz'
            )
            T = current_df['T']
            T = np.transpose(T, [3, 0, 1, 2])
            T = reshape(T, [T.shape[0], -1])
            for k_affinity in xrange(len(affinity_names)):
                current_affinity = affinity_names[k_affinity]
                for k_linkage in xrange(len(linkage)):
                    current_linkage = linkage[k_linkage]
                    clustAlg.linkage = current_linkage
                    if ((current_affinity == 'canberra') or 
                        (current_affinity == 'correlation') or
                        (current_affinity == 'rbf')):
                        clustAlg.affinity = 'precomputed'
                        if current_affinity == 'rbf':
                            D = pairwise_distances(T, metric='euclidean')
                            D = - np.exp(-D)
                        else:
                            D = pairwise_distances(T, metric=current_affinity)
                        pred = clustAlg.fit_predict(D)
                    else:
                        clustAlg.affinity = current_affinity
                        pred = clustAlg.fit_predict(T)
                    result[k_comr, k_indr, k_ind, k_affinity, k_linkage, 0] = adjusted_rand_score(
                        y[indices[k_ind]],
                        pred
                    )
                    result[k_comr, k_indr, k_ind, k_affinity, k_linkage, 1] = adjusted_mutual_info_score(
                        y[indices[k_ind]],
                        pred
                    )
                    result[k_comr, k_indr, k_ind, k_affinity, k_linkage, 2] = fowlkes_mallows_score(
                        y[indices[k_ind]],
                        pred
                    )
                    np.savez_compressed(
                        save_results_dirname+save_results_filename, result=result,
                        affinity_names=affinity_names, indices=indices,
                        linkage=linkage
                    )


### 5.4 Results <a class="anchor" id="5-4-results"></a>

[Up to section](#5-glro) $\quad$
[Back to contents](#Contents)

In [16]:
results_dirname = '../results/clustering/'
results_filename = 'glro_ETH80_ac.npz'

df = np.load(results_dirname+results_filename)
linkageList = df['linkage']
affinityList = df['affinity_names']
commonRanks = 1+np.arange(a.shape[0])
individualRanks = 1+np.arange(a.shape[1])

a = np.mean(df['result'], axis=2)
maxval = a[:, :, :, :, 0].max()
maxind = np.where(a[:, :, :, :, 0] == maxval)
print maxval, maxind
print (
    linkageList[maxind[3][0]], affinityList[maxind[2][0]], commonRanks[maxind[0][0]],
    individualRanks[maxind[1][0]]
)
v1, v2, v3 = a[maxind[0][0], maxind[1][0], maxind[2][0], maxind[3][0]]
print " ARI  AMI  FMS"
print "%.3f %.3f %.3f" % (v1, v2, v3)

0.565152558846 (array([7]), array([2]), array([3]), array([0]))
('complete', 'canberra', 8, 3)
 ARI  AMI  FMS
0.565 0.669 0.629


## 6. Group Tucker-(Lr, 1) decomposition (GTLD) <a class="anchor" id="6-gtld"></a>

- [6.1 Initialization](#6-1-init)
- [6.2 Computations](#6-2-comp)
- [6.3 Clustering](#6-3-clust)
- [6.4 Results](#6-4-results)

[Back to contents](#Contents)

### 6.1 Initialization <a class="anchor" id="6-1-init"></a>

[Up to section](#6-gtld) $\quad$
[Back to contents](#Contents)

In [17]:
transposition = [0, 2, 1, 3]
X = np.transpose(data, transposition)
shapeGTLD = [Nsamples, Npix, Nframes, Ncolors]
X = reshape(X, shapeGTLD)
indices = copy.deepcopy(train_indices)
commonRanks = 1+np.arange(10)
individualRanks = 1+np.arange(10)
sourceModes = [0]

affinity_names = [
    'l1', 'l2', 'cosine', 'canberra', 'correlation', 'rbf'
]
linkage = [
    'complete', 'average'
]

method_name = 'als'
constraint_method = 'projected'
maxitnum = 10

### 6.2 Computations <a class="anchor" id="6-2-comp"></a>

[Up to section](#6-gtld) $\quad$
[Back to contents](#Contents)

In [ ]:
save_model_dirname = '../models/clustering/gtld/'
save_filename_base = 'GTLD_contrasted_ETH80'

try:
    os.makedirs(save_model_dirname)
except:
    print 'Already exist: %s' % (save_model_dirname)


times = np.zeros([len(commonRanks), len(individualRanks), len(indices)])

for k_comr in xrange(len(commonRanks)):
    commonRank = commonRanks[k_comr]
    print '\t %d rank' % (commonRank)
    for k_indr in xrange(len(individualRanks)):
        individualRank = individualRanks[k_indr]
        print '\t\t %d rank' % (individualRank)
        for k_ind in xrange(n_splits):#len(indices)):
            print 'Index %d / %d' % (k_ind+1, len(indices))
            current_index = indices[k_ind]
            contrasting = GTLDContrast(
                n_clusters, individualRank, commonRank, shapeObject=shapeGTLD[1:],
                maxitnum=maxitnum, epsilon=1e-5, sourceModes=sourceModes,
                method=method_name, nShortModes=None, constraintMethod=constraint_method,
                fullModesConstraint=None, modeSizeFirstPriority=True
            )
            
            t1 = time.clock()
            T = contrasting.fit_transform(
                X[current_index], individualRank, commonRank, 
                verbose=True, recover=True
            )
            t2 = time.clock()
            times[k_comr, k_ind] = t2-t1
            save_filename = str(k_ind) + '_' + save_filename_base + '_crank='+str(commonRank)
            save_filename += '_irank='+str(individualRank)
            np.savez_compressed(save_model_dirname+save_filename, T=T, times=times)

### 6.3 Clustering <a class="anchor" id="6-3-clust"></a>

[Up to section](#6-gtld) $\quad$
[Back to contents](#Contents)

In [ ]:
model_dirname = '../models/clustering/gtld/'
model_filename_base = 'GTLD_contrasted_ETH80'

save_results_dirname = '../results/clustering/'
save_results_filename = 'gtld_ETH80_ac'

try:
    os.makedirs(save_results_dirname)
except:
    print 'Already exist: %s' % (save_results_dirname)

result = np.zeros([len(commonRanks), len(individualRanks), n_splits, len(affinity_names), len(linkage), 3])
clustAlg = AgglomerativeClustering(n_clusters=n_clusters)

for k_comr in xrange(len(commonRanks)):
    commonRank = commonRanks[k_comr]
    print '\t %d rank' % (commonRank)
    for k_indr in xrange(len(individualRanks)):
        individualRank = individualRanks[k_indr]
        print '\t\t %d rank' % (individualRank)
        for k_ind in xrange(n_splits):#len(indices)):
            print 'Index %d / %d' % (k_ind+1, len(indices))
            current_index = indices[k_ind]
            current_df = np.load(
                model_dirname+str(k_ind)+'_'+model_filename_base+'_crank='+str(commonRank)+\
                '_irank='+str(individualRank)+'.npz'
            )
            T = current_df['T']
            T = np.transpose(T, [3, 0, 1, 2])
            T = reshape(T, [T.shape[0], -1])
            for k_affinity in xrange(len(affinity_names)):
                current_affinity = affinity_names[k_affinity]
                for k_linkage in xrange(len(linkage)):
                    current_linkage = linkage[k_linkage]
                    clustAlg.linkage = current_linkage
                    if ((current_affinity == 'canberra') or 
                        (current_affinity == 'correlation') or
                        (current_affinity == 'rbf')):
                        clustAlg.affinity = 'precomputed'
                        if current_affinity == 'rbf':
                            D = pairwise_distances(T, metric='euclidean')
                            D = - np.exp(-D)
                        else:
                            D = pairwise_distances(T, metric=current_affinity)
                        pred = clustAlg.fit_predict(D)
                    else:
                        clustAlg.affinity = current_affinity
                        pred = clustAlg.fit_predict(T)
                    result[k_comr, k_indr, k_ind, k_affinity, k_linkage, 0] = adjusted_rand_score(
                        y[indices[k_ind]],
                        pred
                    )
                    result[k_comr, k_indr, k_ind, k_affinity, k_linkage, 1] = adjusted_mutual_info_score(
                        y[indices[k_ind]],
                        pred
                    )
                    result[k_comr, k_indr, k_ind, k_affinity, k_linkage, 2] = fowlkes_mallows_score(
                        y[indices[k_ind]],
                        pred
                    )
                    np.savez_compressed(
                        save_results_dirname+save_results_filename, result=result,
                        affinity_names=affinity_names, indices=indices,
                        linkage=linkage
                    )


### 6.4 Results <a class="anchor" id="6-4-results"></a>

[Up to section](#6-gtld) $\quad$
[Back to contents](#Contents)

In [18]:
results_dirname = '../results/clustering/'
results_filename = 'gtld_ETH80_ac.npz'

df = np.load(results_dirname+results_filename)
linkageList = df['linkage']
affinityList = df['affinity_names']
commonRanks = 1+np.arange(a.shape[0])
individualRanks = 1+np.arange(a.shape[1])

a = np.mean(df['result'], axis=2)
maxval = a[:, :, :, :, 0].max()
maxind = np.where(a[:, :, :, :, 0] == maxval)
print maxval, maxind
print (
    linkageList[maxind[3][0]], affinityList[maxind[2][0]], commonRanks[maxind[0][0]],
    individualRanks[maxind[1][0]]
)
v1, v2, v3 = a[maxind[0][0], maxind[1][0], maxind[2][0], maxind[3][0]]
print " ARI  AMI  FMS"
print "%.3f %.3f %.3f" % (v1, v2, v3)

0.582585887064 (array([7]), array([2]), array([3]), array([0]))
('complete', 'canberra', 8, 3)
 ARI  AMI  FMS
0.583 0.693 0.640
